In [ ]:
# src/Elemento0/get_data.py
import requests
import pandas as pd
import numpy as np
import json
import os

def convert_to_cm(height_list):
    """
    Convierte la altura a centímetros.
    La altura viene como lista: ["6'8", "203 cm"] o ["-"]
    """
    if not height_list or height_list == ["-"]:
        return np.nan
    
    # Tomar el primer elemento que no sea "-"
    for height_str in height_list:
        if height_str != "-":
            height_str = str(height_str).strip()
            
            # Si ya está en cm
            if "cm" in height_str.lower():
                try:
                    return float(height_str.lower().replace("cm", "").strip())
                except:
                    continue
            
            # Si está en pies y pulgadas (formato: "6'8"")
            if "'" in height_str:
                try:
                    # Formato: "6'8""
                    parts = height_str.split("'")
                    feet = float(parts[0])
                    inches_str = parts[1].replace('"', '').strip()
                    inches = float(inches_str) if inches_str else 0
                    return (feet * 30.48) + (inches * 2.54)
                except:
                    continue
            
            # Intentar convertir directamente
            try:
                return float(height_str)
            except:
                continue
    
    return np.nan

def convert_to_kg(weight_list):
    """
    Convierte el peso a kilogramos.
    El peso viene como lista: ["980 lb", "443 kg"] o ["-"]
    """
    if not weight_list or weight_list == ["-"]:
        return np.nan
    
    # Tomar el primer elemento que no sea "-"
    for weight_str in weight_list:
        if weight_str != "-":
            weight_str = str(weight_str).strip()
            
            # Si ya está en kg
            if "kg" in weight_str.lower():
                try:
                    return float(weight_str.lower().replace("kg", "").strip())
                except:
                    continue
            
            # Si está en libras
            if "lb" in weight_str.lower():
                try:
                    lbs = float(weight_str.lower().replace("lb", "").strip())
                    return lbs * 0.453592
                except:
                    continue
            
            # Intentar convertir directamente
            try:
                return float(weight_str)
            except:
                continue
    
    return np.nan

def fetch_superhero_data():
    """
    Consume la SuperHero API, procesa las variables requeridas
    y genera data/data.csv con el dataset final.
    """
    print("Consumiendo SuperHero API...")
    
    # URL de la API
    url = "https://akabab.github.io/superhero-api/api/all.json"
    
    try:
        # Hacer la petición a la API
        response = requests.get(url)
        response.raise_for_status()
        
        data = response.json()
        print(f"Se obtuvieron {len(data)} superhéroes de la API")
        
    except requests.exceptions.RequestException as e:
        print(f"Error al consumir la API: {e}")
        return
    
    processed_data = []
    
    for hero in data:
        try:
            powerstats = hero.get('powerstats', {})
            appearance = hero.get('appearance', {})
            
            # Obtener height y weight como listas (formato original de la API)
            height_list = appearance.get('height', ["-"])
            weight_list = appearance.get('weight', ["-"])
            
            # Convertir unidades
            height_cm = convert_to_cm(height_list)
            weight_kg = convert_to_kg(weight_list)
            
            hero_data = {
                'intelligence': powerstats.get('intelligence', np.nan),
                'strength': powerstats.get('strength', np.nan),
                'speed': powerstats.get('speed', np.nan),
                'durability': powerstats.get('durability', np.nan),
                'combat': powerstats.get('combat', np.nan),
                'height_cm': height_cm,
                'weight_kg': weight_kg,
                'power': powerstats.get('power', np.nan)
            }
            
            processed_data.append(hero_data)
                    
        except Exception as e:
            print(f"Error procesando héroe {hero.get('name', 'Unknown')}: {e}")
            continue
    
    print(f"Se procesaron {len(processed_data)} héroes")
    
    df = pd.DataFrame(processed_data)
    print(f"Dataset final con {len(df)} registros")
    
    # Calcular medias para reemplazar NaN
    height_mean = df['height_cm'].mean()
    weight_mean = df['weight_kg'].mean()
    
    print(f"\nMedias calculadas:")
    print(f"Altura media: {height_mean:.2f} cm")
    print(f"Peso medio: {weight_mean:.2f} kg")
    
    # Reemplazar NaN con las medias
    df['height_cm'].fillna(height_mean, inplace=True)
    df['weight_kg'].fillna(weight_mean, inplace=True)
    
    # Crear directorio data si no existe
    os.makedirs('data', exist_ok=True)
    
    # Guardar el dataset
#    output_path = 'data/data.csv'
 #   df.to_csv(output_path, index=False)
 #   print(f"Dataset guardado en: {output_path}")
    
    # Mostrar información del dataset
    print("\nInformación del dataset:")
    print(f"Columnas: {list(df.columns)}")
    print(f"Forma: {df.shape}")
    
    # Mostrar valores nulos por columna (deberían ser 0 ahora)
    print("\nValores nulos por columna (después de reemplazar con media):")
    null_counts = df.isnull().sum()
    for col, null_count in null_counts.items():
        print(f"  {col}: {null_count} nulos")
    
    print(f"Total de valores nulos: {df.isnull().sum().sum()}")
    
    print("\nPrimeras 10 filas:")
    print(df.head(10))
    
    print("\nEstadísticas descriptivas:")
    print(df.describe())
    
    # Mostrar algunos ejemplos de conversión
    print("\nEjemplos de conversión:")
    sample_heroes = data[:5]
    for i, hero in enumerate(sample_heroes):
        name = hero.get('name', 'Unknown')
        height_orig = hero.get('appearance', {}).get('height', ['-'])
        weight_orig = hero.get('appearance', {}).get('weight', ['-'])
        height_cm = df.iloc[i]['height_cm']
        weight_kg = df.iloc[i]['weight_kg']
        
        print(f"{name}:")
        print(f"  Altura original: {height_orig} -> {height_cm:.2f} cm")
        print(f"  Peso original: {weight_orig} -> {weight_kg:.2f} kg")

if __name__ == "__main__":
    fetch_superhero_data()

In [2]:
# src/orchestrator.py
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

# Cargar y preparar los datos una vez
def load_data():
    """Carga y prepara el dataset para entrenamiento"""
    df = pd.read_csv("C:\\Users\\ejfs2\\gitwars-superheroes-lab10\\data\\data.csv")
    
    # Separar características y variable objetivo
    X = df.drop('power', axis=1)
    y = df['power']
    
    # Dividir en train y test (80-20)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Escalar características (importante para SVM y MLP)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

# Cargar datos una vez al importar el módulo
X_train, X_test, y_train, y_test = load_data()

def evaluate_svm(C, gamma):
    """  
    Entrena un SVM con los hiperparámetros dados  
    y devuelve un float (R² score).
    """  
    try:
        # Crear y entrenar modelo SVM (sin random_state)
        model = SVR(C=C, gamma=gamma)
        model.fit(X_train, y_train)
        
        # Predecir y calcular métrica
        y_pred = model.predict(X_test)
        score = r2_score(y_test, y_pred)
        
        # Asegurar que el score esté en [0,1]
        return max(0.0, float(score))
    
    except Exception as e:
        print(f"Error en evaluate_svm: {e}")
        return 0.0

def evaluate_rf(n_estimators, max_depth):
    """  
    Entrenar un Random Forest y retornar un float  
    con la métrica de desempeño (R²).
    """  
    try:
        # Crear y entrenar modelo Random Forest
        model = RandomForestRegressor(
            n_estimators=int(n_estimators),
            max_depth=None if max_depth == 0 else int(max_depth),
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_train, y_train)
        
        # Predecir y calcular métrica
        y_pred = model.predict(X_test)
        score = r2_score(y_test, y_pred)
        
        return max(0.0, float(score))
    
    except Exception as e:
        print(f"Error en evaluate_rf: {e}")
        return 0.0

def evaluate_mlp(hidden_layer_sizes, alpha):
    """ 
    Entrenar un MLPRegressor y devolver la métrica final (R²).
    """ 
    try:
        # Crear y entrenar modelo MLP
        model = MLPRegressor(
            hidden_layer_sizes=hidden_layer_sizes,
            alpha=alpha,
            random_state=42,
            max_iter=1000,
            early_stopping=True,
            n_iter_no_change=10,
            validation_fraction=0.1
        )
        model.fit(X_train, y_train)
        
        # Predecir y calcular métrica
        y_pred = model.predict(X_test)
        score = r2_score(y_test, y_pred)
        
        return max(0.0, float(score))
    
    except Exception as e:
        print(f"Error en evaluate_mlp: {e}")
        return 0.0

# Script de prueba para verificar que funciona
if __name__ == "__main__":
    print("Probando funciones del orquestador...")
    
    # Probar SVM con parámetros del dominio de BO
    print("\n1. Probando SVM...")
    svm_score = evaluate_svm(C=1.0, gamma=0.1)
    print(f"SVM R² score: {svm_score:.4f}")
    
    # Probar Random Forest con parámetros del dominio de BO
    print("\n2. Probando Random Forest...")
    rf_score = evaluate_rf(n_estimators=50, max_depth=10)
    print(f"Random Forest R² score: {rf_score:.4f}")
    
    # Probar MLP con parámetros del dominio de BO
    print("\n3. Probando MLP...")
    mlp_score = evaluate_mlp(hidden_layer_sizes=(50,), alpha=0.001)
    print(f"MLP R² score: {mlp_score:.4f}")
    
    print("\n✅ Todas las funciones ejecutan correctamente!")
    
    # Probar con múltiples valores
    print("\n--- Pruebas adicionales con diferentes parámetros ---")
    
    print("\nSVM con diferentes parámetros:")
    for C in [0.1, 1.0, 10.0]:
        for gamma in [0.001, 0.01, 0.1]:
            score = evaluate_svm(C=C, gamma=gamma)
            print(f"C={C}, gamma={gamma}: R² = {score:.4f}")
    
    print("\nRandom Forest con diferentes parámetros:")
    for n_est in [10, 50, 100]:
        for depth in [5, 10, 20]:
            score = evaluate_rf(n_estimators=n_est, max_depth=depth)
            print(f"n_estimators={n_est}, max_depth={depth}: R² = {score:.4f}")

Probando funciones del orquestador...

1. Probando SVM...
SVM R² score: 0.3611

2. Probando Random Forest...
Random Forest R² score: 0.4625

3. Probando MLP...
MLP R² score: 0.3803

✅ Todas las funciones ejecutan correctamente!

--- Pruebas adicionales con diferentes parámetros ---

SVM con diferentes parámetros:
C=0.1, gamma=0.001: R² = 0.0000
C=0.1, gamma=0.01: R² = 0.0257
C=0.1, gamma=0.1: R² = 0.0935
C=1.0, gamma=0.001: R² = 0.0305
C=1.0, gamma=0.01: R² = 0.2378
C=1.0, gamma=0.1: R² = 0.3611
C=10.0, gamma=0.001: R² = 0.2487
C=10.0, gamma=0.01: R² = 0.3905
C=10.0, gamma=0.1: R² = 0.4094

Random Forest con diferentes parámetros:
n_estimators=10, max_depth=5: R² = 0.4449
n_estimators=10, max_depth=10: R² = 0.4173
n_estimators=10, max_depth=20: R² = 0.3975
n_estimators=50, max_depth=5: R² = 0.4646
n_estimators=50, max_depth=10: R² = 0.4625
n_estimators=50, max_depth=20: R² = 0.4641
n_estimators=100, max_depth=5: R² = 0.4561
n_estimators=100, max_depth=10: R² = 0.4533
n_estimators=100, 

In [5]:
# src/optimizer.py
import numpy as np
import pandas as pd

# ---
# Kernel RBF
# ---
def rbf_kernel(x1, x2, length_scale=1.0):
    """  
    Implementar kernel RBF  
    """
    x1 = np.array(x1)
    x2 = np.array(x2)
    squared_dist = np.sum((x1 - x2) ** 2)
    return np.exp(-squared_dist / (2 * length_scale ** 2))

# ---
# Ajuste del GP
# ---
def fit_gp(X, y, length_scale=1.0, noise=1e-6):
    """  
    Construir matriz K usando el kernel RBF  
    Resolver (K + noise*I)^(-1) y  
    Regresar los parámetros necesarios para predecir  
    """
    n = len(X)
    K = np.zeros((n, n))
    
    # Construir matriz de kernel
    for i in range(n):
        for j in range(n):
            K[i, j] = rbf_kernel(X[i], X[j], length_scale)
    
    # Agregar ruido a la diagonal
    K_noisy = K + noise * np.eye(n)
    
    # Resolver sistema lineal: K_noisy * alpha = y
    try:
        alpha = np.linalg.solve(K_noisy, y)
    except np.linalg.LinAlgError:
        # Si hay problemas numéricos, usar pseudoinversa
        alpha = np.linalg.pinv(K_noisy) @ y
    
    return alpha, K, X

# ---
# Predicción del GP
# ---
def gp_predict(X_train, y_train, X_test, length_scale=1.0, noise=1e-6):
    """  
    Calcular media mu(x*) y varianza sigma^2(x*)  
    Para cada punto en X_test  
    """
    alpha, K, X_train_arr = fit_gp(X_train, y_train, length_scale, noise)
    
    mu = np.zeros(len(X_test))
    sigma = np.zeros(len(X_test))
    
    for i, x_star in enumerate(X_test):
        # Vector k(x*, X_train)
        k_star = np.array([rbf_kernel(x_star, x_train, length_scale) for x_train in X_train_arr])
        
        # Media predictiva
        mu[i] = k_star @ alpha
        
        # Varianza predictiva
        sigma_sq = rbf_kernel(x_star, x_star, length_scale) - k_star @ np.linalg.solve(K + noise * np.eye(len(K)), k_star)
        sigma[i] = max(0, sigma_sq)  # Asegurar no negatividad
    
    return mu, sigma

# ---
# Función de adquisición UCB
# ---
def acquisition_ucb(mu, sigma, kappa=2.0):
    """  
    UCB = mu + kappa * sigma
    """ 
    return mu + kappa * sigma

# ---
# Dominios discretos de búsqueda (según especificación)
# ---
def get_search_domain(model_name):
    """Retorna el dominio de búsqueda para cada modelo"""
    if model_name == "svm":
        # SVM: C ∈ {0.1, 1, 10, 100}, gamma ∈ {0.001, 0.01, 0.1, 1}
        domain = []
        for C in [0.1, 1, 10, 100]:
            for gamma in [0.001, 0.01, 0.1, 1]:
                domain.append({
                    'params': [C, gamma],
                    'numeric': [C, gamma]
                })
        return domain
    
    elif model_name == "rf":
        # RF: n_estimators ∈ {10, 20, 50, 100}, max_depth ∈ {2, 4, 6, 8}
        domain = []
        for n_est in [10, 20, 50, 100]:
            for depth in [2, 4, 6, 8]:
                domain.append({
                    'params': [n_est, depth],
                    'numeric': [n_est, depth]
                })
        return domain
    
    elif model_name == "mlp":
        # MLP: hidden_layer_sizes ∈ {(16,), (32,), (64,), (32,16)}, alpha ∈ {10^-4, 10^-3, 10^-2}
        domain = []
        hidden_configs = [(16,), (32,), (64,), (32, 16)]
        alphas = [1e-4, 1e-3, 1e-2]
        
        for hidden in hidden_configs:
            for alpha in alphas:
                # Representación numérica para el GP
                num_layers = len(hidden)
                total_neurons = sum(hidden)
                domain.append({
                    'params': (hidden, alpha),
                    'numeric': [num_layers, total_neurons, alpha]
                })
        return domain
    
    else:
        raise ValueError(f"Modelo no soportado: {model_name}")

# ---
# Función de evaluación para cada modelo
# ---
def evaluate_model(model_name, params):
    """Evalúa el modelo con los parámetros dados"""
    if model_name == "svm":
        C, gamma = params
        return evaluate_svm(C=C, gamma=gamma)
    elif model_name == "rf":
        n_estimators, max_depth = params
        return evaluate_rf(n_estimators=n_estimators, max_depth=max_depth)
    elif model_name == "mlp":
        hidden_layer_sizes, alpha = params
        return evaluate_mlp(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha)
    else:
        raise ValueError(f"Modelo no soportado: {model_name}")

# ---
# BO principal
# ---
def optimize_model(model_name, n_init=3, n_iter=10):
    """ 
    Optimización Bayesiana para seleccionar hiperparámetros
    """
    print(f" Iniciando BO para {model_name.upper()}...")
    
    # Obtener dominio de búsqueda
    domain = get_search_domain(model_name)
    print(f"Dominio de búsqueda: {len(domain)} puntos")
    
    # Historia de evaluaciones
    X_evaluated = []  # Representación numérica para GP
    y_evaluated = []  # Scores
    params_evaluated = []  # Parámetros originales
    
    # 1. Samplear puntos iniciales aleatorios
    initial_indices = np.random.choice(len(domain), size=min(n_init, len(domain)), replace=False)
    
    for idx in initial_indices:
        domain_point = domain[idx]
        params = domain_point['params']
        numeric_repr = domain_point['numeric']
        
        score = evaluate_model(model_name, params)
        
        X_evaluated.append(numeric_repr)
        y_evaluated.append(score)
        params_evaluated.append(params)
        
        print(f"  Punto inicial {len(X_evaluated)}: {params} -> R² = {score:.4f}")
    
    best_score = max(y_evaluated)
    best_params = params_evaluated[np.argmax(y_evaluated)]
    
    # 2. Ciclo iterativo de BO
    for iteration in range(n_iter):
        print(f"\n Iteración {iteration + 1}/{n_iter}")
        
        # Ajustar GP con puntos observados
        X_array = np.array(X_evaluated)
        y_array = np.array(y_evaluated)
        
        # Crear array numérico del dominio para predicción
        domain_numeric = np.array([point['numeric'] for point in domain])
        
        # Predecir para todo el dominio
        mu, sigma = gp_predict(X_array, y_array, domain_numeric)
        
        # Calcular UCB
        ucb = acquisition_ucb(mu, sigma, kappa=2.0)
        
        # Seleccionar próximo punto (excluyendo ya evaluados)
        candidate_scores = []
        candidate_indices = []
        
        for i, point in enumerate(domain):
            # Verificar si ya fue evaluado
            already_evaluated = False
            for eval_params in params_evaluated:
                if model_name == "mlp":
                    # Para MLP, comparar tuplas
                    if point['params'][0] == eval_params[0] and abs(point['params'][1] - eval_params[1]) < 1e-6:
                        already_evaluated = True
                        break
                else:
                    # Para SVM y RF, comparar listas
                    if np.allclose(point['params'], eval_params, atol=1e-3):
                        already_evaluated = True
                        break
            
            if not already_evaluated:
                candidate_scores.append(ucb[i])
                candidate_indices.append(i)
        
        if not candidate_indices:
            print("  No hay más puntos por evaluar")
            break
        
        # Seleccionar mejor candidato por UCB
        best_candidate_idx = candidate_indices[np.argmax(candidate_scores)]
        next_domain_point = domain[best_candidate_idx]
        next_params = next_domain_point['params']
        next_numeric = next_domain_point['numeric']
        
        # Evaluar el punto seleccionado
        score = evaluate_model(model_name, next_params)
        
        # Actualizar historia
        X_evaluated.append(next_numeric)
        y_evaluated.append(score)
        params_evaluated.append(next_params)
        
        print(f"  Punto seleccionado: {next_params}")
        print(f"  UCB: {ucb[best_candidate_idx]:.4f}, R² real: {score:.4f}")
        
        # Actualizar mejor global
        if score > best_score:
            best_score = score
            best_params = next_params
            print(f"   NUEVO MEJOR! R² = {best_score:.4f}")
    
    print(f"\n Optimización completada para {model_name.upper()}")
    print(f"Mejores parámetros: {best_params}")
    print(f"Mejor R² score: {best_score:.4f}")
    
    return best_params, best_score

# Script de prueba completo con los 3 modelos
if __name__ == "__main__":
    print(" Probando Optimización Bayesiana con los 3 modelos...")
    
    # Probar con SVM
    print("\n" + "="*60)
    best_params_svm, best_score_svm = optimize_model("svm", n_init=2, n_iter=5)
    print(f"\n Resultado final SVM:")
    print(f"Parámetros: {best_params_svm}")
    print(f"R²: {best_score_svm:.4f}")
    
    # Probar con Random Forest
    print("\n" + "="*60)
    best_params_rf, best_score_rf = optimize_model("rf", n_init=2, n_iter=5)
    print(f"\n Resultado final Random Forest:")
    print(f"Parámetros: {best_params_rf}")
    print(f"R²: {best_score_rf:.4f}")
    
    # Probar con MLP
    print("\n" + "="*60)
    best_params_mlp, best_score_mlp = optimize_model("mlp", n_init=2, n_iter=5)
    print(f"\n Resultado final MLP:")
    print(f"Parámetros: {best_params_mlp}")
    print(f"R²: {best_score_mlp:.4f}")
    
    # Resumen comparativo
    print("\n" + "="*60)
    print(" RESUMEN COMPARATIVO DE OPTIMIZACIÓN BAYESIANA")
    print("="*60)
    print(f"SVM:      R² = {best_score_svm:.4f} - Parámetros: {best_params_svm}")
    print(f"RF:       R² = {best_score_rf:.4f} - Parámetros: {best_params_rf}")
    print(f"MLP:      R² = {best_score_mlp:.4f} - Parámetros: {best_params_mlp}")
    
    best_overall_score = max(best_score_svm, best_score_rf, best_score_mlp)
    best_model = "SVM" if best_score_svm == best_overall_score else "RF" if best_score_rf == best_overall_score else "MLP"
    print(f"\n🏆 MEJOR MODELO: {best_model} con R² = {best_overall_score:.4f}")

 Probando Optimización Bayesiana con los 3 modelos...

 Iniciando BO para SVM...
Dominio de búsqueda: 16 puntos
  Punto inicial 1: [0.1, 0.1] -> R² = 0.0935
  Punto inicial 2: [0.1, 1] -> R² = 0.0051

 Iteración 1/5
  Punto seleccionado: [10, 0.001]
  UCB: 2.0000, R² real: 0.2487
   NUEVO MEJOR! R² = 0.2487

 Iteración 2/5
  Punto seleccionado: [100, 0.001]
  UCB: 2.0000, R² real: 0.3790
   NUEVO MEJOR! R² = 0.3790

 Iteración 3/5
  Punto seleccionado: [100, 1]
  UCB: 1.4929, R² real: 0.2825

 Iteración 4/5
  Punto seleccionado: [10, 1]
  UCB: 1.4138, R² real: 0.3172

 Iteración 5/5
  Punto seleccionado: [1, 0.001]
  UCB: 1.1798, R² real: 0.0305

 Optimización completada para SVM
Mejores parámetros: [100, 0.001]
Mejor R² score: 0.3790

 Resultado final SVM:
Parámetros: [100, 0.001]
R²: 0.3790

 Iniciando BO para RF...
Dominio de búsqueda: 16 puntos
  Punto inicial 1: [50, 6] -> R² = 0.4557
  Punto inicial 2: [50, 2] -> R² = 0.4008

 Iteración 1/5
  Punto seleccionado: [50, 4]
  UCB: 2.

In [7]:
# src/random_search.py
import numpy as np

def random_search(model_name, n_iter=10):
    """
    Implementa Random Search para comparar con BO
    """
    print(f"🎲 Ejecutando Random Search para {model_name.upper()}...")
    
    domain = get_search_domain(model_name)
    best_score = -1
    best_params = None
    history = []
    
    # Realizar búsqueda aleatoria
    for i in range(min(n_iter, len(domain))):
        # Seleccionar punto aleatorio no repetido
        available_indices = [idx for idx in range(len(domain)) if idx not in [h[0] for h in history]]
        
        if not available_indices:
            break
            
        idx = np.random.choice(available_indices)
        domain_point = domain[idx]
        params = domain_point['params']
        
        # Evaluar modelo
        if model_name == "svm":
            score = evaluate_svm(C=params[0], gamma=params[1])
        elif model_name == "rf":
            score = evaluate_rf(n_estimators=params[0], max_depth=params[1])
        elif model_name == "mlp":
            score = evaluate_mlp(hidden_layer_sizes=params[0], alpha=params[1])
        
        history.append((idx, params, score))
        
        # Actualizar mejor
        if score > best_score:
            best_score = score
            best_params = params
            
        print(f"  Iteración {i+1}: {params} -> R² = {score:.4f}")
    
    print(f"✅ Random Search completado - Mejor R²: {best_score:.4f}")
    return best_params, best_score, history